# Image Augmentation Functions

## Library Imports

In [2]:
from PIL import ImageGrab
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output

% pylab inline
% matplotlib inline

ImportError: No module named scenedetect

## Functions

In [ ]:
def resize_img(orig_img, new_dim):
    '''
    DESCRIPTION:
        - resizes the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - new_dim is the base number of pixels for the new image.
    OUTPUT:
        - resized is a numpy array of the resized image.
    '''
    r = float(new_dim) / orig_img.shape[1]
    dim = (new_dim, int(orig_img.shape[0] * r))
    resized = cv2.resize(orig_img, dim, interpolation=cv2.INTER_AREA)
    
    plt.imshow(resized)
    plt.xticks([])
    plt.yticks([])
    
    return resized

In [ ]:
def rotate_img(orig_img, deg_rot, scale):
    '''
    DESCRIPTION:
        - rotates the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - scale (btwn 0 and 1) zooms in on the image. scale (> 1) zooms out on the image. 
        - scale can be used to crop the image based only on the center.
    OUTPUT:
        - rotated_img is a numpy array of the rotated image.
    '''
    (height, width) = orig_img.shape[:2]
    center = (width/2, height/2)
    matrix = cv2.getRotationMatrix2D(center,
                                     angle=deg_rot,
                                     scale=scale)
    rotated_img = cv2.warpAffine(orig_img,
                                 matrix,
                                 (width, height))
    plt.imshow(rotated_img)
    plt.xticks([])
    plt.yticks([])
    
    return rotated_img

In [ ]:
def crop_img(orig_img, h1, h2, w1, w2):
    '''
    DESCRIPTION:
        - crops the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - h1 and h2 defines height
        - w1 and w2 defines the width
    OUTPUT:
        - cropped_img is a numpy array of the cropped image.
    '''
    cropped_img = orig_img[h1:h2, w1:w2]
    
    plt.imshow(cropped_img)
    plt.xticks([])
    plt.yticks([])
    return cropped_img

### Read in the image

In [ ]:
t1 = cv2.imread('data/humansofny/891237_183126561884885_1511388695_n.jpg')
t1 = cv2.cvtColor(t1, cv2.COLOR_BGR2RGB)
print(t1.shape)

### Show the original image

In [ ]:
plt.imshow(t1)
plt.xticks([])
plt.yticks([])

### 1. Reshape the image

#### Define the ratio

In [ ]:
# Around 1/10th of original pixel density
ratio = 100.0 / t1.shape[1]
dim = (100, int(t1.shape[0] * ratio))
resized = cv2.resize(t1, dim, interpolation=cv2.INTER_AREA)

#### Show the resized image

In [ ]:
plt.imshow(resized)
plt.xticks([])
plt.yticks([])

#### Create a function for resizing images

In [ ]:
t = resize_img(t1, int(t1.shape[1]*.1))

### 2. Rotate the image

#### Define the center of image

In [ ]:
(h, w) = t1.shape[:2]
center = (w/2, h/2)

#### Define the rotation

In [ ]:
m = cv2.getRotationMatrix2D(center, angle=45, scale=1)
rotated = cv2.warpAffine(t1, m, (w, h))

#### Show the rotated image

In [ ]:
plt.imshow(rotated)
plt.xticks([])
plt.yticks([])

#### Create function to rotate image

In [ ]:
r_img = rotate_img(t1, deg_rot=210, scale=np.random.uniform(low=1, high=4))

### 3. Crop the image

#### Define the dimensions of the crop

In [ ]:
h1, h2 = 400, 10000
w1, w2 = 600, 10000

crop = t1[h1:h2, w1:w2]

#### Show the cropped image

In [ ]:
plt.imshow(crop)
plt.xticks([])
plt.yticks([])

#### Create a function for cropping images

In [ ]:
c_img = crop_img(t1, h1, h2, w1, w2)

## Test ImageGrab

In [ ]:
t2 = ImageGrab.grab(bbox=None) #bbox specifies specific region (bbox= x,y,width,height *starts top-left)
t2_np = np.array(t2) #this is the array obtained from conversion
t2_frame = cv2.cvtColor(t2_np, cv2.COLOR_BGR2GRAY)
plt.imshow(t2_frame)
plt.xticks([])
plt.yticks([])

In [ ]:
def get_fps(video):
    # Find OpenCV version
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    # With webcam get(CV_CAP_PROP_FPS) does not work.
    # Let's see for ourselves.

    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

In [11]:
def get_frames(file_path):
    # Grab the input device, in this case the webcam
    # You can also give path to the video file
    vid = cv2.VideoCapture(file_path)

    try:
        while(True):
            # Capture frame-by-frame
            ret, frame = vid.read()
            if not ret:
                # Release the Video Device if ret is false
                vid.release()
                # Message to be displayed after releasing the device
                print("Released Video Resource")
                break
            # Convert the image from OpenCV BGR format to matplotlib RGB format
            # to display the image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Turn off the axis
            axis('off')
            # Title of the window
            title("Input Stream")
            # Display the frame
            imshow(frame)
            show()
            # Display the frame until new frame is available
            clear_output(wait=True)

    #         get_fps(vid)

    except KeyboardInterrupt:
        # Release the Video Device
        vid.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")

In [15]:
get_frames("data/humansofamsterdam/10454743_1514322305454692_1807296673_n.mp4")

Released Video Resource


In [2]:
from threading import Thread
import sys
import cv2

if sys.version_info >= (3,0):
    from queue import Queue
else:
    from Queue import Queue

In [4]:
class FileVideoStream:
    def __init__(self, path, queuesize=128):
        self.stream = cv2.VideoCapture(path)
        self.stopped = False
        self.Q = Queue(maxsize=queuesize)
    def start(self):
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        return self
    def update(self):
        try:
            while(True):
                if self.stopped:
                    return
                if not self.Q.full():
                    # Capture frame-by-frame
                    ret, frame = self.stream.read()
                    if not ret:
                        # Release the Video Device if ret is false
                        self.stop()
                        # Message to be displayed after releasing the device
                        print("Released Video Resource")
                        return
                    # Convert the image from OpenCV BGR format to matplotlib RGB format
                    # to display the image
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Turn off the axis
                    axis('off')
                    # Title of the window
                    title("Input Stream")
                    # Display the frame
                    imshow(frame)
                    show()
                    # Display the frame until new frame is available
                    clear_output(wait=True)
                    
                    self.Q.put(frame)

            #         get_fps(vid)

        except KeyboardInterrupt:
            # Release the Video Device
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            
    def more(self):
        return self.Q.qsize() > 0
    def stop(self):
        self.stopped = True

In [25]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import uuid

In [22]:
a = np.array([1,2,3,3])
b = np.array([2000,-0.11,100])

In [20]:
cosine_similarity(a,b)

/home/plim0793/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/plim0793/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.3069376]])

In [35]:
c = [[2],[1]]
if c:
    print(a)

[1 2 3 3]
